# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f450c90c070>
├── 'a' --> tensor([[ 0.5033, -1.0354,  1.0424],
│                   [ 1.0295,  1.0506, -2.3281]])
└── 'x' --> <FastTreeValue 0x7f450c90c9d0>
    └── 'c' --> tensor([[-0.0324, -0.5094, -0.5279,  0.7295],
                        [ 1.0834, -0.3532, -0.4174, -0.6022],
                        [ 0.4785, -2.2128,  0.9645, -0.5289]])

In [4]:
t.a

tensor([[ 0.5033, -1.0354,  1.0424],
        [ 1.0295,  1.0506, -2.3281]])

In [5]:
%timeit t.a

77.8 ns ± 0.212 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f450c90c070>
├── 'a' --> tensor([[-1.6654, -0.8445,  1.0496],
│                   [ 0.8499, -2.4235, -0.6134]])
└── 'x' --> <FastTreeValue 0x7f450c90c9d0>
    └── 'c' --> tensor([[-0.0324, -0.5094, -0.5279,  0.7295],
                        [ 1.0834, -0.3532, -0.4174, -0.6022],
                        [ 0.4785, -2.2128,  0.9645, -0.5289]])

In [7]:
%timeit t.a = new_value

81 ns ± 0.315 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.5033, -1.0354,  1.0424],
               [ 1.0295,  1.0506, -2.3281]]),
    x: Batch(
           c: tensor([[-0.0324, -0.5094, -0.5279,  0.7295],
                      [ 1.0834, -0.3532, -0.4174, -0.6022],
                      [ 0.4785, -2.2128,  0.9645, -0.5289]]),
       ),
)

In [10]:
b.a

tensor([[ 0.5033, -1.0354,  1.0424],
        [ 1.0295,  1.0506, -2.3281]])

In [11]:
%timeit b.a

69.7 ns ± 0.178 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0228,  1.0237, -1.9579],
               [ 0.3323,  0.2582, -0.3524]]),
    x: Batch(
           c: tensor([[-0.0324, -0.5094, -0.5279,  0.7295],
                      [ 1.0834, -0.3532, -0.4174, -0.6022],
                      [ 0.4785, -2.2128,  0.9645, -0.5289]]),
       ),
)

In [13]:
%timeit b.a = new_value

598 ns ± 1.55 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

991 ns ± 7.13 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.7 µs ± 74.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

169 µs ± 483 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

168 µs ± 453 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f450c898af0>
├── 'a' --> tensor([[[ 0.5033, -1.0354,  1.0424],
│                    [ 1.0295,  1.0506, -2.3281]],
│           
│                   [[ 0.5033, -1.0354,  1.0424],
│                    [ 1.0295,  1.0506, -2.3281]],
│           
│                   [[ 0.5033, -1.0354,  1.0424],
│                    [ 1.0295,  1.0506, -2.3281]],
│           
│                   [[ 0.5033, -1.0354,  1.0424],
│                    [ 1.0295,  1.0506, -2.3281]],
│           
│                   [[ 0.5033, -1.0354,  1.0424],
│                    [ 1.0295,  1.0506, -2.3281]],
│           
│                   [[ 0.5033, -1.0354,  1.0424],
│                    [ 1.0295,  1.0506, -2.3281]],
│           
│                   [[ 0.5033, -1.0354,  1.0424],
│                    [ 1.0295,  1.0506, -2.3281]],
│           
│                   [[ 0.5033, -1.0354,  1.0424],
│                    [ 1.0295,  1.0506, -2.3281]]])
└── 'x' --> <FastTreeValue 0x7f450c859880>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

37.7 µs ± 161 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4462c89a00>
├── 'a' --> tensor([[ 0.5033, -1.0354,  1.0424],
│                   [ 1.0295,  1.0506, -2.3281],
│                   [ 0.5033, -1.0354,  1.0424],
│                   [ 1.0295,  1.0506, -2.3281],
│                   [ 0.5033, -1.0354,  1.0424],
│                   [ 1.0295,  1.0506, -2.3281],
│                   [ 0.5033, -1.0354,  1.0424],
│                   [ 1.0295,  1.0506, -2.3281],
│                   [ 0.5033, -1.0354,  1.0424],
│                   [ 1.0295,  1.0506, -2.3281],
│                   [ 0.5033, -1.0354,  1.0424],
│                   [ 1.0295,  1.0506, -2.3281],
│                   [ 0.5033, -1.0354,  1.0424],
│                   [ 1.0295,  1.0506, -2.3281],
│                   [ 0.5033, -1.0354,  1.0424],
│                   [ 1.0295,  1.0506, -2.3281]])
└── 'x' --> <FastTreeValue 0x7f450c898c10>
    └── 'c' --> tensor([[-0.0324, -0.5094, -0.5279,  0.7295],
                        [ 1.0834, -0.3532, -0.4174, -0.6022],
                 

In [23]:
%timeit t_cat(trees)

35.5 µs ± 121 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

65.8 µs ± 74.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.0324, -0.5094, -0.5279,  0.7295],
                       [ 1.0834, -0.3532, -0.4174, -0.6022],
                       [ 0.4785, -2.2128,  0.9645, -0.5289]],
              
                      [[-0.0324, -0.5094, -0.5279,  0.7295],
                       [ 1.0834, -0.3532, -0.4174, -0.6022],
                       [ 0.4785, -2.2128,  0.9645, -0.5289]],
              
                      [[-0.0324, -0.5094, -0.5279,  0.7295],
                       [ 1.0834, -0.3532, -0.4174, -0.6022],
                       [ 0.4785, -2.2128,  0.9645, -0.5289]],
              
                      [[-0.0324, -0.5094, -0.5279,  0.7295],
                       [ 1.0834, -0.3532, -0.4174, -0.6022],
                       [ 0.4785, -2.2128,  0.9645, -0.5289]],
              
                      [[-0.0324, -0.5094, -0.5279,  0.7295],
                       [ 1.0834, -0.3532, -0.4174, -0.6022],
                       [ 0.4785, -2.2128,  0.9645, -0.5289]],

In [26]:
%timeit Batch.stack(batches)

91.3 µs ± 213 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.0324, -0.5094, -0.5279,  0.7295],
                      [ 1.0834, -0.3532, -0.4174, -0.6022],
                      [ 0.4785, -2.2128,  0.9645, -0.5289],
                      [-0.0324, -0.5094, -0.5279,  0.7295],
                      [ 1.0834, -0.3532, -0.4174, -0.6022],
                      [ 0.4785, -2.2128,  0.9645, -0.5289],
                      [-0.0324, -0.5094, -0.5279,  0.7295],
                      [ 1.0834, -0.3532, -0.4174, -0.6022],
                      [ 0.4785, -2.2128,  0.9645, -0.5289],
                      [-0.0324, -0.5094, -0.5279,  0.7295],
                      [ 1.0834, -0.3532, -0.4174, -0.6022],
                      [ 0.4785, -2.2128,  0.9645, -0.5289],
                      [-0.0324, -0.5094, -0.5279,  0.7295],
                      [ 1.0834, -0.3532, -0.4174, -0.6022],
                      [ 0.4785, -2.2128,  0.9645, -0.5289],
                      [-0.0324, -0.5094, -0.5279,  0.7295],
                   

In [28]:
%timeit Batch.cat(batches)

168 µs ± 212 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

389 µs ± 4.14 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
